In [60]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

In [61]:
# with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time

In [62]:
concert_title = []
concert_location = []
concert_location_id = []
concert_date = []
locationId = {}

site = { 1 : "인터파크",
         2 : "멜론티켓",
         3 : "티켓링크"}


with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://tickets.interpark.com/contents/genre/concert")

    # click "콘서트 전체보기"
    all_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/article[1]/section/div/div/div/button[1]')))
    ActionChains(driver).click(all_button).perform()

    # scroll down inifinitely, with a limit of 60 seconds
    SCROLL_PAUSE_TIME = 5
    MAX_SCROLL_DURATION = 10  # 최대 60초 (1분)

    start_time = time.time()
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        current_time = time.time()
        if current_time - start_time > MAX_SCROLL_DURATION:
            print("최대 스크롤 시간 초과 - 중단")
            break

        # scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("더 이상 스크롤할 내용 없음 - 중단")
            break

        last_height = new_height
    # end of the scrolling


    # locate the concert container and append the concert information
    container = driver.find_element(By.XPATH, '//*[@id="contents"]/article[5]/section/div[2]/div[2]')
    a_tags = container.find_elements(By.TAG_NAME, "a")

    for i in range(1, len(a_tags)+1):
        concert_title.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[1]').text)
        concert_location.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[2]').text)
        concert_date.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[3]').text)

최대 스크롤 시간 초과 - 중단


In [63]:
len(concert_title), len(concert_location), len(concert_date)

(25, 25, 25)

In [64]:
concert_date[6].split(' ~ '), concert_date[7].split(' ~ ')
from datetime import datetime
import json

# convert the date format from 'YYYY.MM.DD' to 'YYYY-MM-DD'
def convert_date(raw_date):
    parsed_date = datetime.strptime(raw_date, '%Y.%m.%d').date()
    return parsed_date.strftime('%Y-%m-%d')

# split the date into start and end dates
def split_date(date):
    date_arr = date.split(' ~ ')
    if len(date_arr) == 1:
        return convert_date(date_arr[0]), convert_date(date_arr[0])
    elif len(date_arr) == 2:
        return convert_date(date_arr[0]), convert_date(date_arr[0][:5]+date_arr[1])
    

# update the locationId dictionary and return the location id
def convert_location_id(location):
    locationId[location] = locationId.get(location, len(locationId) + 1)
    return locationId[location]





# convert the concert information into a JSON format
def json_format(title_arr, location_arr, date_arr):
    records = [{
        "title": title,
        "location": convert_location_id(location),
        "start_date": split_date(date)[0],
        "end_date": split_date(date)[1],
        "site": 1
        } for title, location, date in zip(title_arr, location_arr, date_arr)]
    
    return records


#  create concert_info JSON file
data = json_format(concert_title, concert_location, concert_date)
with open('concerts.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# create locationId JSON file
with open('locationId.json', 'w', encoding='utf-8') as f:
    json.dump(locationId, f, ensure_ascii=False, indent=2)


In [56]:
len(locationId), concert_location

(24,
 ['고양종합운동장 주경기장',
  '천안예술의전당 소공연장',
  '올림픽공원',
  '고양종합운동장',
  '엑스코 동관',
  '올림픽공원 88잔디마당, KSPO DOME',
  '잠실 실내체육관',
  '난지한강공원',
  '일산 킨텍스 제1전시장 1홀',
  '인천 남동체육관',
  '창원컨벤션센터',
  '인스파이어 엔터테인먼트 리조트',
  '청주대학교 석우문화체육관',
  '광주여대 유니버시아드 체육관',
  '부산 벡스코 제1전시장',
  '대전컨벤션센터 제2전시장',
  '송도컨벤시아 4홀',
  '안양체육관',
  '경희대학교 국제캠퍼스 선승관',
  '잠실실내체육관',
  '일산 킨텍스 제2전시장 9홀',
  '인스파이어 아레나',
  '올림픽공원',
  '성남아트센터 오페라하우스',
  '킨텍스 제2전시장 10홀'])

In [ ]:

container = driver.find_element(By.XPATH, '//*[@id="contents"]/article[5]/section/div[2]/div[2]')
a_tags = container.find_elements(By.TAG_title, "a")
print(f"총 {len(a_tags)}개의 링크가 발견되었습니다.")

# container
# //*[@id="contents"]/article[5]/section/div[2]/div[2]

#  Format of first concert
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[1] 이름
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[2] 장소In
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[3] 날짜

#  format of second concert
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[2]/ul/li[1]


In [ ]:
"""
Yes24
url = http://ticket.yes24.com/New/Rank/Ranking.aspx?genre=15456

메인페이지에서 스크랩핑 하기에는 공연일자가 나오지 않음.
rank 페이지에서 진행




Best 3
/html/body/div[5]/div[3]/div[1]/a/p[1] 제목1
/html/body/div[5]/div[3]/div[2]/a/p[1] 제목2

/html/body/div[5]/div[3]/div[1]/a/p[2]/text()[1] 공연일자1
/html/body/div[5]/div[3]/div[1]/a/p[2]/text()[2] 장소1

/html/body/div[5]/div[3]/div[2]/a/p[2]/text()[1] 공연일자2
/html/body/div[5]/div[3]/div[2]/a/p[2]/text()[2] 장소2


4위 이후~ 
/html/body/div[5]/div[4]/div[1]/div[3]/p[2]/a   이름1
                          여기다
/html/body/div[5]/div[4]/div[33]/div[3]/p[2]/a  이름2


/html/body/div[5]/div[4]/div[33]/div[4]/p/text()[1] 날짜
/html/body/div[5]/div[4]/div[33]/div[4]/p/text()[2] 장소
                            여기
/html/body/div[5]/div[4]/div[34]/div[4]/p/text()[1] 날짜

"""